In [1]:
from scipy.spatial import distance as dist

In [2]:
pip install --upgrade imutils

Note: you may need to restart the kernel to use updated packages.


In [3]:
from imutils.video import VideoStream

In [4]:
from imutils import face_utils

In [5]:
import numpy as np

In [6]:
import imutils

In [7]:
pip install cmake

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [10]:
import dlib

In [11]:
# then defining the eye aspect ratio
# where we're going to find the distance ratio of vertical & horizontal eye landmark
def eye_asp_ratio(eye):
    # finding the euclidean distance between two points of vertical eye landmark
    point1 = dist.euclidean(eye[1], eye[5])
    point2 = dist.euclidean(eye[2], eye[4])
    
    # finding euclidean distance of horizontal eye
    point3 = dist.euclidean(eye[0], eye[3])
    
    # aspect ratio of eye
    ratio = (point1+point2)/(2.0*point3)# here we are taking average of both vertical length
    # and the dividing it with horizontal length to get ratio
    return ratio

# when we get constant eye aspect ratio- open eye 
# & when decreases to zero suddenly - blink indicated

In [12]:
# defining two constants
# specifying an experimented value(0.3) in eye_ar_thresh_value
# if eye_asp_ratio falls below then the
# number of frames counted when driver closes his/her eyes
eye_ar_thresh_value = 0.30

In [13]:
# number of frames counted when driver closed their eyes
# and if this exceeds this constant value- ALARM WILL SET ON.
eye_ar_consec_frames = 20

In [14]:
# defining counter to count number of frames generated when
# eye_asp_ratio lies below threshold
counter = 0

In [15]:
# now creating facial landmark using dlib's face detector pre trained model 
face_detector = dlib.get_frontal_face_detector()
face_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
# the facial landmark produced by dlib's functions are 
# desired ordered list which extract important features
# from face.
# (http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2)
# above file is trained model for dlib

In [16]:
# by using these indexes, we can easily extract the important
# features of eye region
# extracting the eye feature from the set of facial landmark 
(Startl, Endl) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(Startr, Endr) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [17]:
import cv2 as cvlib

In [18]:
def put_alarm_text(f_cap):
    cvlib.putText(f_cap, "****************HEY! AWAKE****************", (10, 30),
                    cvlib.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cvlib.putText(f_cap, "****************HEY! AWAKE****************", (10,325),
                    cvlib.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    

In [ ]:
cap=cvlib.VideoCapture(0)
flag_val=0
while True:
    ret, frame_cap =cap.read()
    frame_cap = imutils.resize(frame_cap, width=450)
    gray_cap = cvlib.cvtColor(frame_cap, cvlib.COLOR_BGR2GRAY)
    captures = face_detector(gray_cap, 0)
    for capture in captures:
        shape_val = face_predictor(gray_cap, capture)
        shape_val = face_utils.shape_to_np(shape_val)#converting to NumPy Array
        lEye = shape_val[Startl:Endl]
        rEye = shape_val[Startr:Endr]
        lEAR = eye_asp_ratio(lEye)
        rEAR = eye_asp_ratio(rEye)
        ear = (lEAR + rEAR) / 2.0
        lEyeHull = cvlib.convexHull(lEye)
        rEyeHull = cvlib.convexHull(rEye)
        cvlib.drawContours(frame_cap, [lEyeHull], -1, (0, 255, 0), 1)
        cvlib.drawContours(frame_cap, [rEyeHull], -1, (0, 255, 0), 1)
        if ear < eye_ar_thresh_value:
            flag_val += 1
            if flag_val >= eye_ar_consec_frames:
                put_alarm_text(frame_cap)
        else:
            flag_val = 0
    cvlib.imshow("Frame", frame_cap)
    key = cvlib.waitKey(1) & 0xFF
    if key == ord("q"):
        cvlib.destroyAllWindows()
        cap.release()
        break